# S08_T01_Feature_Engineering

### Ex1: Agafa un conjunt de dades de tema esportiu que t'agradi i normalitza els atributs categòrics en dummy. Estandaritza els atributs numèrics amb StandardScaler

In [2]:
#Importem llibreries necessàries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import ttest_1samp,ttest_ind,f_oneway

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


In [ ]:
#Hem d'analitzar les nostres dades, i tenir clar que és el que volem extreure. P.ex, en el nostre cas podem estudiar 
#a partir d'alguns dels atributs físics quins esportistes o atletes tenen més facilitat o probabilitat de guanyar medalles;
#o basant-nos en el pais d'origen o en l'sport a q es dediquen...
#Per poder portar a terme un estudi d'aquests, utilitzem els models de Machine Learning!
#Ens demanen normalitzar els atributs categòrics en dummy, i estandaritzar els numèrics amb Standard Scaler, 
#amb el que reduïrem el nostre dataframe amb els atributs que triarem per poder comparar i fer l'estudi.

In [11]:
#Agafem el mateix dataset del Sprint anterior 120 years of Olympic history athletes and results

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
statistics_df= pd.read_csv("athlete_events.csv")
statistics_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [53]:
new_statistics_df = statistics_df[["Name", "Sex", "Age", "Height", "Weight", "NOC", "Season", "City", "Sport", "Medal"]].copy()
new_statistics_df.head()

,Name,Sex,Age,Height,Weight,NOC,Season,City,Sport,Medal
0,A Dijiang,M,24.0,180.0,80.0,CHN,Summer,Barcelona,Basketball,0
1,A Lamusi,M,23.0,170.0,60.0,CHN,Summer,London,Judo,0
2,Gunnar Nielsen Aaby,M,24.0,0.0,0.0,DEN,Summer,Antwerpen,Football,0
3,Edgar Lindenau Aabye,M,34.0,0.0,0.0,DEN,Summer,Paris,Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,Winter,Calgary,Speed Skating,0


In [54]:
#Comprovem quins tipus de variables hi ha al nostre dataset
new_statistics_df.dtypes

Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
NOC        object
Season     object
City       object
Sport      object
Medal      object
dtype: object

In [47]:
#Comprovem si tenim NaN values i si hi ha els reemplaçem per zero's
new_statistics_df.fillna(value=0, inplace = True)
new_statistics_df.head()

,Name,Sex,Age,Height,Weight,NOC,Season,City,Sport,Medal
0,A Dijiang,M,24.0,180.0,80.0,CHN,Summer,Barcelona,Basketball,0
1,A Lamusi,M,23.0,170.0,60.0,CHN,Summer,London,Judo,0
2,Gunnar Nielsen Aaby,M,24.0,0.0,0.0,DEN,Summer,Antwerpen,Football,0
3,Edgar Lindenau Aabye,M,34.0,0.0,0.0,DEN,Summer,Paris,Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,Winter,Calgary,Speed Skating,0


Pandas has a function which can turn a categorical variable into a series of zeros and ones, which makes them a lot easier to quantify and compare: get_dummies

In [19]:
dummy_Sex = pd.get_dummies(new_statistics_df["Sex"]) #convertim la columna "Sex" de categorical variable a dummy variable
dummy_Sex.head()

,F,M
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [20]:
dummy_Season = pd.get_dummies(new_statistics_df["Season"]) #convertim la columna "Season" de categorical variable a dummy variable
dummy_Season.head()

,Summer,Winter
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1


In [49]:
#convertim la columna "Medal" de categorical variable a dummy variable
dummy_Medal = pd.get_dummies(new_statistics_df["Medal"], drop_first = True)
dummy_Medal.head()

,Bronze,Gold,Silver
0,0,0,0
1,0,0,0
2,0,0,0
3,0,1,0
4,0,0,0


In [21]:
dummy_NOC = pd.get_dummies(new_statistics_df["NOC"]) #convertim la columna "NOC" de categorical variable a dummy variable
dummy_NOC.head()

,AFG,AHO,ALB,ALG,AND,ANG,ANT,ANZ,ARG,ARM,ARU,ASA,AUS,AUT,AZE,BAH,BAN,BAR,BDI,BEL,BEN,BER,BHU,BIH,BIZ,BLR,BOH,BOL,BOT,BRA,BRN,BRU,BUL,BUR,CAF,CAM,CAN,CAY,CGO,CHA,CHI,CHN,CIV,CMR,COD,COK,COL,COM,CPV,CRC,CRO,CRT,CUB,CYP,CZE,DEN,DJI,DMA,DOM,ECU,EGY,ERI,ESA,ESP,EST,ETH,EUN,FIJ,FIN,FRA,FRG,FSM,GAB,GAM,GBR,GBS,GDR,GEO,GEQ,GER,GHA,GRE,GRN,GUA,GUI,GUM,GUY,HAI,HKG,HON,HUN,INA,IND,IOA,IRI,IRL,IRQ,ISL,ISR,ISV,ITA,IVB,JAM,JOR,JPN,KAZ,KEN,KGZ,KIR,KOR,KOS,KSA,KUW,LAO,LAT,LBA,LBR,LCA,LES,LIB,LIE,LTU,LUX,MAD,MAL,MAR,MAS,MAW,MDA,MDV,MEX,MGL,MHL,MKD,MLI,MLT,MNE,MON,MOZ,MRI,MTN,MYA,NAM,NBO,NCA,NED,NEP,NFL,NGR,NIG,NOR,NRU,NZL,OMA,PAK,PAN,PAR,PER,PHI,PLE,PLW,PNG,POL,POR,PRK,PUR,QAT,RHO,ROT,ROU,RSA,RUS,RWA,SAA,SAM,SCG,SEN,SEY,SGP,SKN,SLE,SLO,SMR,SOL,SOM,SRB,SRI,SSD,STP,SUD,SUI,SUR,SVK,SWE,SWZ,SYR,TAN,TCH,TGA,THA,TJK,TKM,TLS,TOG,TPE,TTO,TUN,TUR,TUV,UAE,UAR,UGA,UKR,UNK,URS,URU,USA,UZB,VAN,VEN,VIE,VIN,VNM,WIF,YAR,YEM,YMD,YUG,ZAM,ZIM
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [100]:
new_statistics_df = pd.concat([new_statistics_df, dummy_Sex, dummy_Season, dummy_Medal, dummy_NOC], axis=1)
new_statistics_df.head()

,Name,Sex,Age,Height,Weight,NOC,Season,City,Sport,Medal,F,M,Summer,Winter,Bronze,Gold,Silver,AFG,AHO,ALB,ALG,AND,ANG,ANT,ANZ,ARG,ARM,ARU,ASA,AUS,AUT,AZE,BAH,BAN,BAR,BDI,BEL,BEN,BER,BHU,BIH,BIZ,BLR,BOH,BOL,BOT,BRA,BRN,BRU,BUL,BUR,CAF,CAM,CAN,CAY,CGO,CHA,CHI,CHN,CIV,CMR,COD,COK,COL,COM,CPV,CRC,CRO,CRT,CUB,CYP,CZE,DEN,DJI,DMA,DOM,ECU,EGY,ERI,ESA,ESP,EST,ETH,EUN,FIJ,FIN,FRA,FRG,FSM,GAB,GAM,GBR,GBS,GDR,GEO,GEQ,GER,GHA,GRE,GRN,GUA,GUI,GUM,GUY,HAI,HKG,HON,HUN,INA,IND,IOA,IRI,IRL,IRQ,ISL,ISR,ISV,ITA,IVB,JAM,JOR,JPN,KAZ,KEN,KGZ,KIR,KOR,KOS,KSA,KUW,LAO,LAT,LBA,LBR,LCA,LES,LIB,LIE,LTU,LUX,MAD,MAL,MAR,MAS,MAW,MDA,MDV,MEX,MGL,MHL,MKD,MLI,MLT,MNE,MON,MOZ,MRI,MTN,MYA,NAM,NBO,NCA,NED,NEP,NFL,NGR,NIG,NOR,NRU,NZL,OMA,PAK,PAN,PAR,PER,PHI,PLE,PLW,PNG,POL,POR,PRK,PUR,QAT,RHO,ROT,ROU,RSA,RUS,RWA,SAA,SAM,SCG,SEN,SEY,SGP,SKN,SLE,SLO,SMR,SOL,SOM,SRB,SRI,SSD,STP,SUD,SUI,SUR,SVK,SWE,SWZ,SYR,TAN,TCH,TGA,THA,TJK,TKM,TLS,TOG,TPE,TTO,TUN,TUR,TUV,UAE,UAR,UGA,UKR,UNK,URS,URU,USA,UZB,VAN,VEN,VIE,VIN,VNM,WIF,YAR,YEM,YMD,YUG,ZAM,ZIM
0,A Dijiang,M,-0.084665,0.593263,0.793490,CHN,Summer,Barcelona,Basketball,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A Lamusi,M,-0.212206,0.457117,0.175852,CHN,Summer,London,Judo,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Gunnar Nielsen Aaby,M,-0.084665,-1.857369,-1.677062,DEN,Summer,Antwerpen,Football,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Edgar Lindenau Aabye,M,1.190742,-1.857369,-1.677062,DEN,Summer,Paris,Tug-Of-War,Gold,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Christine Jacoba Aaftink,F,-0.467287,0.661337,0.855253,NED,Winter,Calgary,Speed Skating,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


L'objectiu d'aquesta transformació és poder comparar les variables categòriques amb les numèriques!
Hem transformat les variables categòriques a numeriques amb valor 0 o 1
Ara el nostre dataset està preparat per poder comparar les diferents variables, i poder trobar així les relacions i valors estadístics entre elles.

In [12]:
#Estandaritza els atributs numèrics amb StandardScaler
#Els atributs numèrics que hi ha al nostre dataset són l'edat, alçada i pes

In [101]:
#Obtenim el nostre dataframe amb els valors dels atributs numèrics estandaritzats i els atributs categòrics normalitzats com a dummies
#si fessim només el SatndardScaler, no ens retornaria més que un amatriu, però d'aquesta forma  assignant els valors de nou en el dataframe 
#aconseguim reemplaçar les dades per les dades amb els valors escalats o estandaritzats:
new_statistics_df[["Age", "Height", "Weight"]] = StandardScaler().fit_transform(new_statistics_df[["Age", "Height", "Weight"]])
new_statistics_df.head()

,Name,Sex,Age,Height,Weight,NOC,Season,City,Sport,Medal,F,M,Summer,Winter,Bronze,Gold,Silver,AFG,AHO,ALB,ALG,AND,ANG,ANT,ANZ,ARG,ARM,ARU,ASA,AUS,AUT,AZE,BAH,BAN,BAR,BDI,BEL,BEN,BER,BHU,BIH,BIZ,BLR,BOH,BOL,BOT,BRA,BRN,BRU,BUL,BUR,CAF,CAM,CAN,CAY,CGO,CHA,CHI,CHN,CIV,CMR,COD,COK,COL,COM,CPV,CRC,CRO,CRT,CUB,CYP,CZE,DEN,DJI,DMA,DOM,ECU,EGY,ERI,ESA,ESP,EST,ETH,EUN,FIJ,FIN,FRA,FRG,FSM,GAB,GAM,GBR,GBS,GDR,GEO,GEQ,GER,GHA,GRE,GRN,GUA,GUI,GUM,GUY,HAI,HKG,HON,HUN,INA,IND,IOA,IRI,IRL,IRQ,ISL,ISR,ISV,ITA,IVB,JAM,JOR,JPN,KAZ,KEN,KGZ,KIR,KOR,KOS,KSA,KUW,LAO,LAT,LBA,LBR,LCA,LES,LIB,LIE,LTU,LUX,MAD,MAL,MAR,MAS,MAW,MDA,MDV,MEX,MGL,MHL,MKD,MLI,MLT,MNE,MON,MOZ,MRI,MTN,MYA,NAM,NBO,NCA,NED,NEP,NFL,NGR,NIG,NOR,NRU,NZL,OMA,PAK,PAN,PAR,PER,PHI,PLE,PLW,PNG,POL,POR,PRK,PUR,QAT,RHO,ROT,ROU,RSA,RUS,RWA,SAA,SAM,SCG,SEN,SEY,SGP,SKN,SLE,SLO,SMR,SOL,SOM,SRB,SRI,SSD,STP,SUD,SUI,SUR,SVK,SWE,SWZ,SYR,TAN,TCH,TGA,THA,TJK,TKM,TLS,TOG,TPE,TTO,TUN,TUR,TUV,UAE,UAR,UGA,UKR,UNK,URS,URU,USA,UZB,VAN,VEN,VIE,VIN,VNM,WIF,YAR,YEM,YMD,YUG,ZAM,ZIM
0,A Dijiang,M,-0.084665,0.593263,0.793490,CHN,Summer,Barcelona,Basketball,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A Lamusi,M,-0.212206,0.457117,0.175852,CHN,Summer,London,Judo,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Gunnar Nielsen Aaby,M,-0.084665,-1.857369,-1.677062,DEN,Summer,Antwerpen,Football,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Edgar Lindenau Aabye,M,1.190742,-1.857369,-1.677062,DEN,Summer,Paris,Tug-Of-War,Gold,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Christine Jacoba Aaftink,F,-0.467287,0.661337,0.855253,NED,Winter,Calgary,Speed Skating,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Ex2: Continua amb el conjunt de dades de tema esportiu que t'agradi i aplica l'anàlisi de components principals

### Ex2:Continua amb el conjunt de dades de tema esportiu que t'agradi i normalitza les dades tenint en compte els outliers

In [ ]:
#Random Forest scikit learn algorithm with list of all parameters:
#RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False,class_weight=None)